# update4 fix display_name

In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime, timezone
import json
from tqdm import tqdm
from pathlib import Path
import gzip

In [4]:
import pandas as pd
import numpy as np

In [5]:
from app import db
from models import Source, ISSNtoISSNL
from sqlalchemy import text
from sqlalchemy.orm import Load
from sqlalchemy.exc import MultipleResultsFound

140027311251840: loading merged_into_institutions_dict
140027311251840: loading valid concept IDs
140027311251840: loading valid topic IDs


In [6]:
%%time
df_issnl_file = pd.read_sql_query('select * from issn_audit_20240321.issn_ic_datafile_expanded_202402', db.engine)

CPU times: user 1.16 s, sys: 277 ms, total: 1.44 s
Wall time: 3.28 s


In [9]:
df_issnl_file = df_issnl_file[['valid_issns', 'issnl', 'cluster_title']]

In [10]:
print(len(df_issnl_file))
df_issnl_file.dropna(inplace=True)
print(len(df_issnl_file))
df_issnl_file.drop_duplicates(inplace=True)
print(len(df_issnl_file))


215085
213487
213303


/tmp/ipykernel_2253/4201960640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issnl_file.dropna(inplace=True)
/tmp/ipykernel_2253/4201960640.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issnl_file.drop_duplicates(inplace=True)


In [12]:
df_issnl_file['valid_issns'].duplicated().sum()

0

In [13]:
df_issnl_file['issnl'].duplicated().sum()

72414

In [14]:
len(df_issnl_file[['issnl', 'cluster_title']])

213303

In [16]:
len(df_issnl_file[['issnl', 'cluster_title']].drop_duplicates())

140889

In [19]:
df_issnl_names = df_issnl_file[['issnl', 'cluster_title']].drop_duplicates()
df_issnl_names.set_index('issnl', verify_integrity=True, inplace=True)
len(df_issnl_names)

140889

In [26]:
%%time
try:
    num_updated = 0
    num_not_updated = 0
    for issnl, title in tqdm(df_issnl_names['cluster_title'].items(), total=len(df_issnl_names)):
        updated_date = datetime.now(timezone.utc).isoformat()
        try:
            source = db.session.query(Source).options(Load(Source).lazyload('*')).filter_by(issn=issnl).filter_by(merge_into_id=None).one_or_none()
            if source:
                source_id = source.id
                old_display_name = source.display_name

                source.display_name = title
                source.updated_date = updated_date
                db.session.add(source)

                sq = """INSERT INTO issn_audit_20240321.update4
                    (issnl, old_display_name, new_display_name, updated_date, source_id)
                    VALUES(:issnl, :old_display_name, :new_display_name, :updated_date, :source_id);"""
                db.session.execute(text(sq), {
                    'issnl': issnl,
                    'old_display_name': old_display_name,
                    'new_display_name': title,
                    'updated_date': updated_date,
                    'source_id': source_id,
                })
                num_updated += 1
                if num_updated > 0 and num_updated % 10000 == 0:
                    db.session.commit()
            else:
                num_not_updated += 1
        except MultipleResultsFound:
            num_not_updated += 1
    db.session.commit()
    print(f"{num_updated} updated. {num_not_updated} not updated")
finally:
    db.session.close()

100%|██████████| 140889/140889 [2:20:08<00:00, 16.76it/s]    


135511 updated. 5378 not updated
CPU times: user 4min 45s, sys: 19.2 s, total: 5min 4s
Wall time: 2h 21min 46s


In [27]:
source.display_name

DetachedInstanceError: Instance <Source at 0x7f5a2c299e50> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: http://sqlalche.me/e/14/bhk3)

In [28]:

title

'Soviet journal of communications technology & electronics'

In [6]:
%%time
sq = """select journal_id, display_name, issn, issns, type, country_code, alternate_titles, publisher_id
    from mid.journal
    where merge_into_id is null"""
df_midjournal = pd.read_sql_query(sq, db.engine)

CPU times: user 1.43 s, sys: 66 ms, total: 1.49 s
Wall time: 2.35 s


In [ ]:
%%time
sq = """select journal_id, display_name, issn, issns, type, country_code, alternate_titles, publisher_id
    from mid.journal
    where merge_into_id is null"""
df_midjournal = pd.read_sql_query(sq, db.engine)

CPU times: user 1.43 s, sys: 66 ms, total: 1.49 s
Wall time: 2.35 s


In [7]:
df_midjournal.dropna(subset='issn', inplace=True)

In [8]:
df_midjournal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148415 entries, 1 to 261660
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   journal_id        148415 non-null  int64  
 1   display_name      148415 non-null  object 
 2   issn              148415 non-null  object 
 3   issns             148415 non-null  object 
 4   type              148290 non-null  object 
 5   country_code      109641 non-null  object 
 6   alternate_titles  118625 non-null  object 
 7   publisher_id      67149 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 10.2+ MB


In [9]:
df_midjournal['issn'].duplicated().sum()

4260

In [13]:
df_midjournal['no_issnl_found'] = ~(df_midjournal['issn'].isin(df_issnl['issn'].values))
df_midjournal['no_issnl_found'].value_counts()

no_issnl_found
False    144547
True       3868
Name: count, dtype: int64

In [14]:
df_midjournal[df_midjournal['no_issnl_found']]

,journal_id,display_name,issn,issns,type,country_code,alternate_titles,publisher_id,no_issnl_found
82,4391422086,Proceedings of the Association for Japanese Li...,1531-5533,"[""1531-5533""]",journal,None,None,NaN,True
112,4391633437,Co-Catalyst Journal of Science Education Resea...,3026-1597,"[""3026-1597""]",journal,None,None,4.310315e+09,True
239,4391422089,Nihonkai Cetology,0918-3930,"[""0918-3930"", ""2435-3760""]",journal,None,None,NaN,True
250,4391633469,Metrik Serial Teknologi dan Sains,2774-2989,"[""2774-2989""]",journal,None,None,NaN,True
251,4391633474,MANTRA Jurnal Sastra Indonesia (Sastra Bahasa ...,2987-8969,"[""2987-8969""]",journal,None,None,NaN,True
...,...,...,...,...,...,...,...,...,...
259612,4210178947,Proceedings of the Aristotelian Society (Hardb...,0066-7372,"[""0066-7372"",""0066-7373""]",journal,None,[],4.310321e+09,True
260162,4386621720,Les cahiers du numérique,1469-3380,"[""1469-3380""]",journal,None,None,NaN,True
260183,4306524739,Protokolle zur Bibel,1996-0042,"[""2412-2467""]",journal,None,[],NaN,True
260811,4387287589,Vjesnik dalmatinskih arhiva (Online),2757-0932,"[""2757-0932"", ""2806-8459""]",journal,None,None,NaN,True


In [19]:
df_midjournal[df_midjournal['issn'].duplicated(keep=False)].sort_values('issn')

,journal_id,display_name,issn,issns,type,country_code,alternate_titles,publisher_id
54910,4306504665,Boletín de la Academia colombiana,0001-3773,"[""0001-3773""]",journal,None,[],NaN
136919,4306504666,Boletín de la Academia Colombiana,0001-3773,"[""0001-3773""]",journal,None,[],NaN
46084,4306500404,Acta medica Philippina,0001-6071,"[""0001-6071"", ""2094-9278""]",journal,None,[],4.310320e+09
79744,4210231482,Acta Medica Philippina,0001-6071,"[""2094-9278"",""0001-6071""]",journal,PH,[The National Health Science journal],4.310320e+09
60458,4306500432,Acta Obstetrica et Gynaecologica Japonica,0001-6330,"[""0001-6330""]",journal,None,[],NaN
...,...,...,...,...,...,...,...,...
91498,4387277837,Parasites Hosts and Diseases,2982-5164,"[""2982-5164"", ""2982-6799""]",journal,None,None,4.310319e+09
102645,4387281090,International Journal of Psychiatric Trainees,3005-3870,"[""3005-3870"", ""2957-4080""]",journal,None,None,NaN
78775,4387292761,International Journal of Psychiatric Trainees,3005-3870,"[""3005-3870""]",journal,None,None,NaN
89862,50342286,Yearbook of Pulmonary Disease,8756-3452,"[""8756-3452""]",journal,None,[],4.310321e+09


In [20]:
%%time
# Source IDs that have already been through the updates should be protected
sq = """select source_id from issn_audit_20240321.update1 where reverted is false;"""
df1 = pd.read_sql_query(sq, db.engine)

sq = """select source_id from issn_audit_20240321.update2 where reverted is false;"""
df2 = pd.read_sql_query(sq, db.engine)

df_protected = pd.concat([df1, df2])

CPU times: user 77.9 ms, sys: 0 ns, total: 77.9 ms
Wall time: 837 ms


In [22]:
df_protected

,source_id
0,4210234080
1,4387277910
2,4306499931
3,4210177666
4,4210223324
...,...
8854,4393920245
8855,4393920246
8856,4393920247
8857,4393920248


In [28]:
df_midjournal['is_protected'] = df_midjournal['journal_id'].isin(df_protected['source_id'].values)

In [30]:
df_midjournal['is_protected'].value_counts()

is_protected
True     134124
False     14291
Name: count, dtype: int64

In [6]:
%%time
sq = """select * from issn_audit_20240321.issn_ic_datafile_202402"""
df_datafile = pd.read_sql_query(sq, db.engine)

CPU times: user 599 ms, sys: 127 ms, total: 726 ms
Wall time: 1.98 s


In [7]:
# get dict of issnl to issn
x = df_datafile.dropna(subset=['issnl'])
x = x[x['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
x = x[['issn', 'issnl']]
issnl_to_issn = {}
ignore = [
    '0000-0000',
]
x = x[~(x['issnl'].isin(ignore))]
for name, gbdf in x.groupby('issnl'):
    if name in ignore:
        continue
    issnl_to_issn[name] = gbdf['issn'].tolist()
len(issnl_to_issn)

140984

In [17]:
issn = '0001-5113'
try:
    source = db.session.query(Source).options(Load(Source).lazyload('*')).filter_by(issn=issn).all()
    if source is None:
        source = db.session.query(Source).options(Load(Source).lazyload('*')).filter(Source.issns.contains(issn)).all()
finally:
    db.session.close()


In [18]:
source

[<Source ( http://localhost:5007/S86406033?apiurls ) 86406033 Acta Adriatica>,
 <Source ( http://localhost:5007/S2764458511?apiurls ) 2764458511 Acta Adriatica: International Journal of Marine Sciences>]